# Corpusanalyse UiTdatabank

We hebben de volgende bibliotheken nodig:

In [66]:
from pandas import read_excel, read_csv, DataFrame, Series, concat
from datetime import datetime
from codecs import open
from re import compile
from json import dumps
from datetime import datetime
from random import sample
from collections import Counter

Dan lezen we lezen de gegevens in

In [3]:
df_podium = read_excel("ruwe data/podium.xlsx", sheetname='theaterdans1014')
df_bk = read_excel("ruwe data/beeldendekunsten.xlsx", sheetname='UitRapport')
df_muziek = read_excel("ruwe data/muziek.xlsx", sheetname='Int nat reg')

Voor de muziekgegevens moeten we nog controleren dat concerten eventuele herhalingen hebben, door in de speelmomenten kolom na te gaan wat de speelmomenten zijn. Dit is niet nodig bij de beeldende kunsten, aangezien daar de unit of analysis de tentoonstelling is, onafhankelijk van hoelang die tentoonstelling loopt. Bij podiumkunsten zijn de speelmomenten manueel gecheckt, ocharme simon.

In [4]:
datumregex = compile(r"\d\d/\d\d/\d\d")
df_muziek_expanded = df_muziek.copy()
for row in df_muziek_expanded.iterrows():
    speelmomenten = row[1]["Speelmomenten"]
    if str(speelmomenten) != "nan":    
        for speelmoment in datumregex.findall(speelmomenten):
            speelmoment_dt = datetime(int("20" + speelmoment.split("/")[2]), 
                                      int(speelmoment.split("/")[1]), 
                                      int(speelmoment.split("/")[0]))
            if speelmoment_dt != row[1]["Datum"]:
                df_muziek_expanded = df_muziek_expanded.append(
                    Series(
                        {"Discipline": row[1]["Discipline"],
                         "Subdiscipline": row[1]["Subdiscipline"],
                         "Tekst": row[1]["Tekst"],
                         "Datum": speelmoment_dt,
                         "Gemeente": row[1]["Gemeente"]
                        }, name=speelmoment_dt.isoformat() + " " + str(row[0])
                    )
                )

Voor de tentoonstellingen moeten we ook nog inperken op thema, en ook de permanente tentoonstellingen eruit zwieren.

In [5]:
subdisciplines = ["Beeldhouwkunst", 
                  "Fotografie", 
                  "Grafiek", 
                  "Installatiekunst", 
                  "Kunst en kunsteducatie", 
                  "Meerdere kunstvormen", 
                  "Schilderkunst"]
df_bk_filtered = df_bk[df_bk["Datum tot"] != datetime(1900, 1, 1)]
df_bk_filtered = df_bk_filtered[df_bk_filtered["Subdiscipline"].isin(subdisciplines)]
df_bk_filtered = df_bk_filtered[
    (df_bk_filtered["Datum van"].between(datetime(2014, 1, 1), datetime(2014, 12, 31))) |
    (df_bk_filtered["Datum tot"].between(datetime(2014, 1, 1), datetime(2014, 12, 31)))
]
df_bk_filtered["Datum"] = df_bk_filtered["Datum van"]
df_bk_filtered = df_bk_filtered.drop(["Datum van", "Datum tot"], axis=1)

Even kijken hoe dit er allemaal uitziet:

In [10]:
df_bk_filtered.head()

,Discipline,Tekst,Gemeente,Titel,Subdiscipline,Organisator,Datum
1,Tentoonstelling,Tentoonstelling keramiek van' de negen' afgest...,Hasselt,de ne9en,Meerdere kunstvormen,Stedelijke Academie voor Schone Kunsten Hasselt,2014-11-13
2,Tentoonstelling,Winterexpositie met werken van oa Panamaren...,Kemzeke (Stekene),Noli Me Tangere,Meerdere kunstvormen,Verbeke Foundation,2014-12-10
6,Tentoonstelling,"Project gebaseerd op technologie, instinct en ...",Brussel,Fragments #43-44,Installatiekunst,PointCulture Bruxelles,2013-12-17
8,Tentoonstelling,‘’Tentoonstelling – Tuin - Serres - Vrije Teke...,Gent,Tentoonstelling Modelstudies in Plantentuin UGent,Schilderkunst,NaN,2014-05-08
10,Tentoonstelling,In het kader van Kunst in het Gemeentehuis ste...,Vaalbeek (Oud-Heverlee),Kunst in het gemeentehuis: Lejon-art 70,Schilderkunst,Gemeente Oud-Heverlee,2014-09-24


In [11]:
df_muziek_expanded.head()

,Discipline,Tekst,Datum,Gemeente,Speelmomenten,Subdiscipline,Titel,Organisator
0,Concert,De aangrijpende film die internationaal hoge t...,2014-12-21 00:00:00,Brussel,zo 21/12/14 van 20:00 tot 22:30,Pop en rock,The Broken Circle Breakdown Bluegrass Band,Ancienne Belgique
1,Concert,Zelfs heel wat toppianisten deinzen ervoor ter...,2014-03-20 00:00:00,Antwerpen 18 (Antwerpen),do 20/03/14 van 20:00 tot 22:00,Thema onbepaald,National Philharmonic Of Russia Olv. Vladimir ...,deSingel
2,Concert,“Brilliant young piano trio.” (The Daily Teleg...,2014-10-23 00:00:00,Brussel,do 23/10/14 van 20:00 tot 22:30,Pop en rock,Gogo Penguin: Where Aphex Twin meets Jazz,Ancienne Belgique
3,Concert,"Het Salagon Quartett, waarvan niemand minder d...",2014-03-28 00:00:00,Brugge,vrij 28/03/14 om 20:00,Klassieke muziek,SALAGON QUARTETT / MOZART & BEETHOVEN,Concertgebouw Brugge
4,Concert,De Romeo’s hebben tal van nummer 1-hits op hun...,2014-10-31 00:00:00,Merksem (Antwerpen),vrij 31/10/14 om 20:00,Pop en rock,Feestival,NaN


In [12]:
df_podium.head()

,Tekst,Gemeente,Datum,Discipline,Subdiscipline,Titel,Typologie organisator
0,Met Gil Grainlune.,Brussel,2010-01-12,podium,Theatervoorstelling,Raconte nous des histoires Mr Orts..,CA
1,"Regie David Michels en Bernard Lefrancq, met R...",Brussel,2010-01-31,podium,Theatervoorstelling,La revue 2010,FRG
2,"Van M. de Ghelderode, bewerking José Géal en ...",Brussel,2010-01-02,podium,Theatervoorstelling,La Nativité et le massacre des innocents,LOT
3,"Van Jean-Michel Ribes, regie Eric De Staercke,...",Brussel,2010-01-03,podium,Theatervoorstelling,Théâtre sans animaux,FRG
4,"Van Georges Courteline, regie Valéry Massion, ...",Brussel,2010-01-09,podium,Theatervoorstelling,Faire le malin est le propre de tout imbécile,FRG


We gaan nu de de organisator omzetten naar een typering. We moeten eerst beginnen met die mapping van organisator naar typering op te bouwen op basis van de gegevens die Simon wist aan te leveren.

In [61]:
typering_podium = read_excel("extra gegevens/typologie_organisatoren.xlsx", sheetname='podium')
typering_muziek = read_excel("extra gegevens/typologie_organisatoren.xlsx", sheetname='muziek2')
typering_beeldende = read_excel("extra gegevens/typologie_organisatoren.xlsx", sheetname='beeldende1')

In [91]:
onmapbaar = []

def map_organisator_naar_typologie(item, mapping):
    try:
        return mapping[mapping["key"].str.lower() == str(item).lower()]["value"].values[0]
    except IndexError:
        onmapbaar.append(item)

df_podium["typering"] = df_podium["Typologie organisator"].apply(map_organisator_naar_typologie, args=(typering_podium,))
df_muziek_expanded["typering"] = df_muziek_expanded["Organisator"].apply(map_organisator_naar_typologie, args=(typering_muziek,))
df_bk_filtered["typering"] = df_bk_filtered["Organisator"].apply(map_organisator_naar_typologie, args=(typering_beeldende,))

Nu voegen we alles mooi samen, selecteren enkel de juiste kolommen, en gooien ook duplicaten op basis van datum, gemeente en tekst eruit. Bovendien hebben we de kolom met speelmomenten ook niet meer nodig.

In [93]:
df_muziek_expanded.head()

,Discipline,Tekst,Datum,Gemeente,Speelmomenten,Subdiscipline,Titel,Organisator,typering
0,Concert,De aangrijpende film die internationaal hoge t...,2014-12-21 00:00:00,Brussel,zo 21/12/14 van 20:00 tot 22:30,Pop en rock,The Broken Circle Breakdown Bluegrass Band,Ancienne Belgique,None
1,Concert,Zelfs heel wat toppianisten deinzen ervoor ter...,2014-03-20 00:00:00,Antwerpen 18 (Antwerpen),do 20/03/14 van 20:00 tot 22:00,Thema onbepaald,National Philharmonic Of Russia Olv. Vladimir ...,deSingel,None
2,Concert,“Brilliant young piano trio.” (The Daily Teleg...,2014-10-23 00:00:00,Brussel,do 23/10/14 van 20:00 tot 22:30,Pop en rock,Gogo Penguin: Where Aphex Twin meets Jazz,Ancienne Belgique,None
3,Concert,"Het Salagon Quartett, waarvan niemand minder d...",2014-03-28 00:00:00,Brugge,vrij 28/03/14 om 20:00,Klassieke muziek,SALAGON QUARTETT / MOZART & BEETHOVEN,Concertgebouw Brugge,Vlaams gesubsidieerde kunsten
4,Concert,De Romeo’s hebben tal van nummer 1-hits op hun...,2014-10-31 00:00:00,Merksem (Antwerpen),vrij 31/10/14 om 20:00,Pop en rock,Feestival,NaN,None


In [94]:
df_muziek_expanded_clean = df_muziek_expanded.drop(["Speelmomenten", "Organisator"], axis=1)
df_bk_filtered_clean = df_bk_filtered.drop("Organisator", axis=1)
df_podium_clean = df_podium.drop("Typologie organisator", axis=1)
df = concat([df_podium_clean, df_bk_filtered_clean, df_muziek_expanded_clean])
df = df[df["Datum"].between(datetime(2014, 1, 1), datetime(2014, 12, 31))]
df.drop_duplicates(subset=["Datum", "Titel", "Discipline", "Subdiscipline", "Gemeente", "Tekst"], inplace=True)
df.drop("Titel", axis=1, inplace=True)

We kunnen kort inspecteren hoe deze data eruitzien.

In [95]:
df.head()

,Datum,Discipline,Gemeente,Subdiscipline,Tekst,typering
747,2014-04-02 00:00:00,podium,Brussel,Dansvoorstelling,Shen Yun brengt 5.000 jaar Chinese beschaving ...,Rest
748,2014-04-03 00:00:00,podium,Brussel,Dansvoorstelling,Shen Yun brengt 5.000 jaar Chinese beschaving ...,Rest
749,2014-04-04 00:00:00,podium,Brussel,Dansvoorstelling,Shen Yun brengt 5.000 jaar Chinese beschaving ...,Rest
750,2014-04-05 00:00:00,podium,Brussel,Dansvoorstelling,Shen Yun brengt 5.000 jaar Chinese beschaving ...,Rest
752,2014-04-06 00:00:00,podium,Brussel,Dansvoorstelling,Shen Yun brengt 5.000 jaar Chinese beschaving ...,Rest


We zien dat elke lijn een event beschrijving bevat, de plaats waar een event plaatsvindt, en ook de datum. Merk op dat events die op meerdere dagen plaatsvinden een aparte lijn krijgen. We zullen hiermee rekening houden in de interpretatie van de resultaten.

Telling per discipline van het aantal unieke events:

In [9]:
df["Discipline"].value_counts()

Concert            20500
podium             19406
Tentoonstelling     3511
Name: Discipline, dtype: int64

En hoeveel daarvan hebben geen beschrijving?

In [120]:
len(df[df["Tekst"].str.strip() == ""]["Tekst"])

3416

Zo, we kunnen deze dataset nu mooi uitschrijven naar een Excel bestand, zodat Simon nog enkele laatste correcties en aanvullingen kan doorvoeren.

In [123]:
df.to_csv("samengevoegde data/dataset.csv")

Voor onze analyse hebben we ook nood aan een lijst van namen van landen, coordinaten voor de plaatsnamen, en ook een manueel gemaakte mapping om de plaatsnamen in de UiTdatabank gegevens te normaliseren.

In [98]:
typering = read_csv("extra gegevens/mapping_udb-gemeente_fusie-gemeente.csv", delimiter=';')
coord = read_csv("extra gegevens/coordinaten.csv", delimiter=';')
landen = read_excel("extra gegevens/landen.xlsx", sheetname="uitgebreide lijst 2014")

Laten we even in detail deze tabellen bekijken. De landen:

In [11]:
landen.head()

,Continent (staatkundig),Land,Soort mention,Mention
0,Azië,Afghanistan,korte landnaam,Afghanistan
1,Azië,Afghanistan,inwoner,Afghaan
2,Azië,Afghanistan,inwoner,Afghanen
3,Azië,Afghanistan,adjectief,Afghaans
4,Azië,Afghanistan,adjectief,Afghaanse


Voor elk land weten we in welk (staatkundig) continent het ligt, en we hebben in de kolom 'Mention' verschillende manieren waarop dat land kan voorkomen in de tekst.

Bij typering zien we het volgende:

In [12]:
typering.tail()

,Gemeente Origineel,Postcode,Fusiegemeente,Stedelijkheid fusiegemeenten,Gemeente cluster,Province (English),Country (English),Provincie,Stedelijkheid
931,Kwaremont (Kluisbergen),NaN,Kluisbergen,Niet-stedelijk,Kluisbergen,East Flanders,Belgium,Oost-Vlaanderen,Niet-stedelijk
932,Kleine-Brogel (Peer),NaN,Peer,Niet-stedelijk,Peer,Limburg,Belgium,Limburg,Niet-stedelijk
933,Meilegem (Zwalm),NaN,Zwalm,Niet-stedelijk,Zwalm,East Flanders,Belgium,Oost-Vlaanderen,Niet-stedelijk
934,Beek (Bree),NaN,Bree,Niet-stedelijk,Bree,Limburg,Belgium,Limburg,Niet-stedelijk
935,Oetingen (Gooik),NaN,Gooik,Niet-stedelijk,Gooik,Flemish Brabant,Belgium,Vlaams-Brabant,Niet-stedelijk


De kolom Gemeente Origineel is de naam van de gemeente in de uitdatabank gegevens, en we kunnen de naam in de kolom Fusiegemeente en Province (English) gebruiken om een genormaliseerd zicht te krijgen.

Tot slot hebben we nog de coordinaten:

In [13]:
coord.head()

,Fusiegemeente,adress,latitude,longitude
0,Aalst,"Aalst, Belgium","50,9378101","4,0409517"
1,Aalter,"Aalter, Belgium","51,087349","3,448371"
2,Aarschot,"Aarschot, Belgium","50,9859959","4,8365218"
3,Aartselaar,"Aartselaar, Belgium","51,1340539","4,3844742"
4,Affligem,"Affligem, Belgium","50,9084","4,11281"


Hiermee kunnen we voor iedere Fusiegemeente (zie vorige tabel) de latitude en longitude ophalen.

Nu gaan we voor ieder event in de UiTdatabankgegevens na welk land er vermeld wordt in de beschrijving van dat event. We houden ook al onmiddellijk bij wat de genormaliseerde naam is van de gemeente en de coordinaten van het centrum. Bovendien tellen we binnen het land ook nog de verschillende disciplines.

In [101]:
data = {}
niet_vlaams = ["Jodoigne", "Tournai", "Escanaffles", "Houffalize", "Haulchin", 
               "Braine l'Alleud", "Tourinnes-la-Grosse", "Liège", "Marchienne-au-Pont", "Eupen"]
count = 1
for row in list(df.iterrows())[0:100]:
    if count % 10000 == 0:
        print(count, "of", len(df.index))
    count += 1
    tekst = row[1]["Tekst"]
    gemeente = row[1]["Gemeente"]
    organisatie = row[1]["typering"]
    if str(gemeente) != "nan" and str(gemeente) not in niet_vlaams:
        for land in set(landen["Land"].values):
            regex = compile(r"\b(" + r"|".join(landen[landen["Land"] == land]["Mention"]) + r")\b")
            amount_of_mentions = len(regex.findall(tekst))
            if amount_of_mentions > 0:
                try:
                    typeringlijn = typering[typering["Gemeente Origineel"] == gemeente]
                    fusiegemeente = typeringlijn["Fusiegemeente"].values[0]
                    provincie = typeringlijn["Province (English)"].values[0]
                    lat = coord[coord["Fusiegemeente"] == fusiegemeente]["latitude"].values[0]
                    lon = coord[coord["Fusiegemeente"] == fusiegemeente]["longitude"].values[0]
                    continent = landen[landen["Land"] == land]["Continent (staatkundig)"].values[0]
                    discipline = row[1]["Discipline"]
                    subdiscipline = row[1]["Subdiscipline"]
                    if fusiegemeente not in data:
                        data[fusiegemeente] = {"latitude": float(lat.replace(',', '.')),
                                               "longitude": float(lon.replace(',', '.')),
                                               "provincie": provincie,
                                               "vermeldingen": {}
                                               }
                    if organisatie not in data[fusiegemeente]["vermeldingen"]:
                        data[fusiegemeente]["vermeldingen"][organisatie] = {}
                    if continent not in data[fusiegemeente]["vermeldingen"][organisatie]:
                        data[fusiegemeente]["vermeldingen"][organisatie][continent] = {}
                    if land not in data[fusiegemeente]["vermeldingen"][organisatie][continent]:
                        data[fusiegemeente]["vermeldingen"][organisatie][continent][land] = {}
                    if discipline not in data[fusiegemeente]["vermeldingen"][organisatie][continent][land]:
                        data[fusiegemeente]["vermeldingen"][organisatie][continent][land][discipline] = {}
                    if subdiscipline not in data[fusiegemeente]["vermeldingen"][organisatie][continent][land][discipline] :
                        data[fusiegemeente]["vermeldingen"][organisatie][continent][land][discipline][subdiscipline] = 0
                    data[fusiegemeente]["vermeldingen"][organisatie][continent][land][discipline][subdiscipline] += amount_of_mentions
                except Exception as e:
                    print(gemeente, "geeft een probleem:", e)

We kunnen al eens gauw de structuur van deze gegevens nakijken voor bijvoorbeeld de gemeente Lanaken:

In [33]:
print(dumps(data["Lanaken"], indent=2))

{
  "latitude": 50.8919647,
  "longitude": 5.6572276,
  "provincie": "Limburg",
  "vermeldingen": {
    "Europa": {
      "Belgi\u00eb": {
        "podium": {
          "Theatervoorstelling": 2
        },
        "Concert": {
          "Klassieke muziek": 2
        }
      },
      "Verenigd Koninkrijk": {
        "Concert": {
          "Pop en rock": 2,
          "Klassieke muziek": 5
        }
      },
      "Duitsland": {
        "podium": {
          "Theatervoorstelling": 2
        },
        "Concert": {
          "Dance": 1,
          "Amusementsmuziek": 1
        }
      },
      "Itali\u00eb": {
        "Concert": {
          "Klassieke muziek": 1
        }
      },
      "Frankrijk": {
        "Concert": {
          "NaN": 2,
          "Klassieke muziek": 1
        }
      },
      "Vlaanderen": {
        "podium": {
          "Theatervoorstelling": 2
        },
        "Concert": {
          "NaN": 1
        }
      },
      "Nederland": {
        "podium": {
          "Thea

We kunnen nu deze gegevens samenballen in een tabel.

In [102]:
lines = []
for fusiegemeente in data:
    lat = data[fusiegemeente]["latitude"]
    lon = data[fusiegemeente]["longitude"]
    prov = data[fusiegemeente]["provincie"]
    for typering in data[fusiegemeente]["vermeldingen"]:
        for continent in data[fusiegemeente]["vermeldingen"][organisatie]:
            for land in data[fusiegemeente]["vermeldingen"][organisatie][continent]:
                for discipline in data[fusiegemeente]["vermeldingen"][organisatie][continent][land]:
                    for subdiscipline in data[fusiegemeente]["vermeldingen"][organisatie][continent][land][discipline]:
                        lines.append([fusiegemeente, lat, lon, prov, continent, land, discipline, subdiscipline, typering,
                                      data[fusiegemeente]["vermeldingen"][organisatie][continent][land][discipline][subdiscipline]])
res = DataFrame(lines, columns=['Gemeente', 'Latitude', 'Longitude', 'Provincie', 'Continent', 'Land', 'Discipline', 'Subdiscipline', 'Typering', 'Frequentie'])

De tabel bevat dus de volgende informatie:

In [103]:
res.head()

,Gemeente,Latitude,Longitude,Provincie,Continent,Land,Discipline,Subdiscipline,Typering,Frequentie
0,Brussel,50.85034,4.35171,Brussels,Europa,België,podium,Theatervoorstelling,Lokaal (cultuur)beleid,1
1,Brussel,50.85034,4.35171,Brussels,Europa,Letland,podium,Theatervoorstelling,Lokaal (cultuur)beleid,1
2,Brussel,50.85034,4.35171,Brussels,Europa,België,podium,Theatervoorstelling,Vlaams gesubsidieerde kunsten,1
3,Brussel,50.85034,4.35171,Brussels,Europa,Letland,podium,Theatervoorstelling,Vlaams gesubsidieerde kunsten,1
4,Brussel,50.85034,4.35171,Brussels,Europa,België,podium,Theatervoorstelling,Onderwijs,1


We schrijven tot slot deze tabel uit naar een CSV bestand voor verdere analyse.

In [36]:
res.to_csv("landsvermeldingen.csv")